In [1]:
import tensorflow as tf
print(tf.__version__)

2.5.0


In [2]:
# image data preprocessing
image_size = (224,224)

batch_size = 16


class_mode = 'binary' # for multi-class classification problem, use: class_mode = 'category' 

In [3]:
import pandas as pd
train_df=pd.read_csv('D:/Kaggle/Is a Block Structure Stable or Unstable/blocks-labels.csv',dtype=str)



def append_ext(fn):
    return fn+".jpg"

train_df["id"] = train_df["id"].apply(append_ext)

In [4]:
# change contract of each images using contrast_stretching method
# https://github.com/sakares/histogram-equalization/blob/master/hist-equalize.ipynb
'''import numpy as np
from skimage import data, img_as_float, exposure, io

for img_path in train_df["id"]:
    # Load images
    img = io.imread(f"D:/Kaggle/Is a Block Structure Stable or Unstable/train/{img_path}")

    # Contrast stretching
    p2, p98 = np.percentile(img, (2, 98))
    img_rescale = exposure.rescale_intensity(img, in_range=(p2, p98))
    io.imsave(f"D:/Kaggle/Is a Block Structure Stable or Unstable/contrast_stretching/{img_path}", img_rescale)'''

'import numpy as np\nfrom skimage import data, img_as_float, exposure, io\n\nfor img_path in train_df["id"]:\n    # Load images\n    img = io.imread(f"D:/Kaggle/Is a Block Structure Stable or Unstable/train/{img_path}")\n\n    # Contrast stretching\n    p2, p98 = np.percentile(img, (2, 98))\n    img_rescale = exposure.rescale_intensity(img, in_range=(p2, p98))\n    io.imsave(f"D:/Kaggle/Is a Block Structure Stable or Unstable/contrast_stretching/{img_path}", img_rescale)'

In [5]:
train_data_directory = 'D:/Kaggle/Is a Block Structure Stable or Unstable/contrast_stretching/'

In [6]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    fill_mode='nearest',
    validation_split=0.33 # set validation split
    ) 

train_generator = train_datagen.flow_from_dataframe(
    train_df,
    directory = "D:/Kaggle/Is a Block Structure Stable or Unstable/contrast_stretching/",
    x_col = "id",
    y_col = "stable",
    target_size=image_size,
    batch_size=batch_size,
    class_mode=class_mode, # for multi-class classification problem, use 'category'
    subset='training') # set as training data

validation_generator = train_datagen.flow_from_dataframe(
    train_df,
    directory = "D:/Kaggle/Is a Block Structure Stable or Unstable/contrast_stretching/",
    x_col = "id",
    y_col = "stable",
    target_size=image_size,
    batch_size=batch_size,
    class_mode=class_mode, # for multi-class classification problem, use 'category'
    subset='validation') # set as validation data

Found 34355 validated image filenames belonging to 2 classes.
Found 16921 validated image filenames belonging to 2 classes.


In [7]:
# build a model
base_model = tf.keras.applications.resnet50.ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

base_model.trainable = True

# Create the model
model = tf.keras.Sequential()
model.add(base_model)
model.add(tf.keras.layers.GlobalAveragePooling2D())
model.add(tf.keras.layers.Dense(1024, activation='relu'))
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dense(32, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Functional)        (None, 7, 7, 2048)        23587712  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 1024)              2098176   
_________________________________________________________________
dense_1 (Dense)              (None, 128)               131200    
_________________________________________________________________
dense_2 (Dense)              (None, 32)                4128      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 33        
Total params: 25,821,249
Trainable params: 25,768,129
Non-trainable params: 53,120
_______________________________________

In [8]:
# compile model
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              metrics=tf.keras.metrics.BinaryAccuracy())

# Callbacks
checkpoint = tf.keras.callbacks.ModelCheckpoint(f"D:/Kaggle/Is a Block Structure Stable or Unstable/model/ResNet50_original_photo_stable.h5", 
                             monitor='val_loss', 
                             verbose=1, 
                             save_best_only=True, 
                             save_weights_only=False, 
                             mode='auto', 
                             save_freq='epoch')
early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                      min_delta=0.0001, 
                      patience=3, 
                      verbose=1,
                      restore_best_weights=True,
                      mode='auto')
learning_rate_reduction = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss',
                                            patience=1,
                                            verbose=1,
                                            factor=0.1,
                                            min_lr=0.0000000001)

In [9]:
# fit/train model

nb_epochs = 1000

history = model.fit(
    train_generator,
    steps_per_epoch = train_generator.samples // batch_size,
    validation_data = validation_generator, 
    validation_steps = validation_generator.samples // batch_size,
    epochs = nb_epochs,
    callbacks=[checkpoint, early, learning_rate_reduction])

Epoch 1/1000


C:\Users\Jenny\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\keras\backend.py:5016: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  warnings.warn(


2147/2147 [==============================] - 205s 92ms/step - loss: 0.3303 - binary_accuracy: 0.8243 - val_loss: 0.1158 - val_binary_accuracy: 0.9538

Epoch 00001: val_loss improved from inf to 0.11577, saving model to D:/Kaggle/Is a Block Structure Stable or Unstable/model\ResNet50_stable_or_not.h5


C:\Users\Jenny\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


Epoch 2/1000
2147/2147 [==============================] - 197s 92ms/step - loss: 0.0828 - binary_accuracy: 0.9692 - val_loss: 0.0698 - val_binary_accuracy: 0.9707

Epoch 00002: val_loss improved from 0.11577 to 0.06983, saving model to D:/Kaggle/Is a Block Structure Stable or Unstable/model\ResNet50_stable_or_not.h5
Epoch 3/1000
2147/2147 [==============================] - 196s 91ms/step - loss: 0.0534 - binary_accuracy: 0.9814 - val_loss: 0.0518 - val_binary_accuracy: 0.9823

Epoch 00003: val_loss improved from 0.06983 to 0.05185, saving model to D:/Kaggle/Is a Block Structure Stable or Unstable/model\ResNet50_stable_or_not.h5
Epoch 4/1000
2147/2147 [==============================] - 196s 91ms/step - loss: 0.0346 - binary_accuracy: 0.9879 - val_loss: 0.0327 - val_binary_accuracy: 0.9877

Epoch 00004: val_loss improved from 0.05185 to 0.03270, saving model to D:/Kaggle/Is a Block Structure Stable or Unstable/model\ResNet50_stable_or_not.h5
Epoch 5/1000
2147/2147 [======================

In [13]:
import matplotlib.pyplot as plt

# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')

plt.show()

# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

KeyError: 'acc'